In [ ]:
from aicsimageio import AICSImage
import napari
from aicsimageio.readers import CziReader
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load the image
reader = CziReader("extern_Synlab_2156_17_3_MTB.czi")
# Get whole image
smear = reader.get_image_data("MYX", C=0)


In [ ]:
# save in a new variable the information regarding the 673th tile out of 1345
img = smear[999]
img.shape

print(img.shape)
print(np.max(img))
print(np.min(img))




In [ ]:
# rescale the image to 0-255
def rescale(image: np.ndarray):
    """
    :param image: to  be rescaled
    :return: rescaled image
    """
    return (image - np.min(image)) / (np.max(image) - np.min(image)) * 255

# visualize the image with napari using its numpy array
def visualize_napari(numpy_img: np.ndarray):
    """
    :param numpy_img: image to be visualized
    """
    with napari.gui_qt():
        viewer = napari.Viewer()
        viewer.add_image(numpy_img)

# visualize different images in the same moment
def visualize_all_list_napari(numpy_img_list: np.ndarray):
    """
    :param numpy_img_list: list containing different images to be visualized
    """
    with napari.gui_qt():
        viewer = napari.Viewer()
        for img in numpy_img_list:
            viewer.add_image(img)

# plot histogram of pixel intensity
def plot_histogram(image: np.ndarray):
    plt.hist(image.ravel(),image.max(),[0,image.max()])
    plt.show()


In [ ]:
# different thresholding methods example from opencv documentation.

# global thresholding
ret1,th1 = cv.threshold(img,5000,255,cv.THRESH_BINARY)
# manually setting the global threshold value therefore not automatic / the optimal value / the optimal choice
# Otsu's thresholding
ret2,th2 = cv.threshold(img,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
# Otsu's thresholding after Gaussian filtering
blur = cv.GaussianBlur(img,(5,5),0)
# blur is not really helping us here because we are looking for the opposite task.
ret3,th3 = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
# plot all the images and their histograms
images = [img, 0, th1,
          img, 0, th2,
          blur, 0, th3]
titles = ['Original Noisy Image','Histogram','Global Thresholding (v=127)',
          'Original Noisy Image','Histogram',"Otsu's Thresholding",
          'Gaussian filtered Image','Histogram',"Otsu's Thresholding"]
for i in range(3):
    plt.subplot(3,3,i*3+1),plt.imshow(images[i*3],'gray')
    plt.title(titles[i*3]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+2),plt.hist(images[i*3].ravel(),256)
    plt.title(titles[i*3+1]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+3),plt.imshow(images[i*3+2],'gray')
    plt.title(titles[i*3+2]), plt.xticks([]), plt.yticks([])
plt.show()

#visualize_napari(th1)
img = img.astype('uint8') #sbagliato
#other thresholding methods
img_blur = cv.medianBlur(img,5)
ret,th1 = cv.threshold(img_blur,127,255,cv.THRESH_BINARY)
th2 = cv.adaptiveThreshold(img_blur,255,cv.ADAPTIVE_THRESH_MEAN_C,\
            cv.THRESH_BINARY,11,2)
th3 = cv.adaptiveThreshold(img_blur,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv.THRESH_BINARY,11,2)
titles = ['Original Image', 'Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [img_blur, th1, th2, th3]

In [ ]:
# compute Otsu's thresholding
def otsu_thresholding(image : np.ndarray):
    """
    Threshold and binarize an image using Otsu's method

    :param image: image you want to threshold
    :return: ret: threshold value
              th: binary image
    """
    ret,th = cv.threshold(image,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    #ret is the computed value of the threshold AND th is the image with the threshold applied
    return ret, th

#compute hard thresholding
def hard_thresholding(image : np.ndarray, threshold : int):
    """
    Implement an hard threshold. Take everything above "threshold" to be white
    and everything below "threshold" to be black

    :param image: image to be thresholded
    :param threshold: hard threshold to be implemented
    :return: ret: threshold value
              th: binary image
    """
    ret,th = cv.threshold(image,threshold,255,cv.THRESH_BINARY)
    return ret, th

# approach for going through the different tiles and applying the thresholding separately
# split the whole images into tiles
def split_into_tiles(image : np.ndarray, tile_size: int):
    """
    split image into tiles of shape tile_size*tile_size

    :param image: image to be split
    :param tile_size: dimensions of single tiles
    :return: tiles: list with the different tiles
    """
    tiles = []
    for i in range(0, image.shape[0], tile_size):
        for j in range(0, image.shape[1], tile_size):
            tile = image[i:i+tile_size, j:j+tile_size]
            tiles.append(tile)
    return tiles

# approach in which we sharp (in stead of blur as the example) the image before applying the thresholding
# sharpen the image using a high-pass filter TODO: can we do this better? sharp out better maybe in sub-images?
def sharpen(image: np.ndarray):
    """
    :param image: image to be sharpened
    :return: sharp image
    """
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    return cv.filter2D(image, -1, kernel)

# change np array to uint8 ATENTION: in uint8 only numbers until 2^7 can be stored
def to_uint8(image: np.ndarray):
    return np.uint8(image)


def find_coordinates(image: np.ndarray):
    """
    Find the coordinates of the connected compenents of the image.
    to be more specific, find first coordinate of a component

    :param image: image where we want to find the connected components
    :return: coordinates: array with coordinates of the components in the rows
    """
    num_conn_comp, labels_conn_comp = cv.connectedComponents(image)
    #num_conn_comp tells us how many components we have
    #labels_conn_comp is an image where the entries where there should be
    # a component are the label of that component
    # overwritten in such a way that the the values of the pixels are the value of the corresponding connected component.
    labels_conn_comp = labels_conn_comp.astype(np.uint64)    #cannot use uint8 cannot rappresent all numbers with uint8

    #save coordinates in two dimensional array
    coordinates = np.zeros((num_conn_comp,2),dtype="int64")

    # iterate over the connected components and add the coordinates of the pixels to the list of coordinates
    for i in range(1, num_conn_comp):
        coordinates[i,0] = np.where(labels_conn_comp == i)[0][0]
        coordinates[i,1] = np.where(labels_conn_comp == i)[1][0]

    return coordinates

#connectedComponentsWithStats works better, can get centroid, in this way can put the box/rectangle around the bacilli
#but not all conncected components are identified, might be a problem of the image we give him
def get_connected_components_coordinate(img):
    connectivity = 8
    #find connected components
    num_labels, labels_im, stats, centroids = cv.connectedComponentsWithStats(img, connectivity)
    #get coordinates of connected components
    coordinates = np.zeros((num_labels, 2),dtype=np.uint64)  #NO uint8

    for i in range(1, num_labels):
        coordinates[i,0] = centroids[i,0]
        coordinates[i,1] = centroids[i,1]
    print(coordinates)
    return coordinates

# add the 2d bounding boxes to the image
def add_bounding_boxes(image, coordinates):
    """
    Add whhite rectangles around bacilli

    :param image: image with bacilli to be boxed
    :param coordinates:  coordinates of the center of the bacillus
    """
    for i in range(len(coordinates)):
            x_min = coordinates[i][0]
            x_max = coordinates[i][0]
            y_min = coordinates[i][1]
            y_max = coordinates[i][1]
            cv.rectangle(image, (y_min+15, x_min+15), (y_max-15, x_max-15), (255, 0, 0), 4)


#reconstruct image from different tiles given the number of tiles in x and y direction and a list of tiles
def reconstruct_image(tiles: list, x_tiles: int, y_tiles: int):
    """
    :param tiles:    list with the different single tiles
    :param x_tiles:  how many tiles fit in the x axis
    :param y_tiles:  how many tiles fit in the y axis
    :return:         numpy array, reconstructed image
    """
    big_image = np.zeros((x_tiles*tiles[0].shape[0], y_tiles*tiles[0].shape[1]))
    for i in range(x_tiles):
        for j in range(y_tiles):
            big_image[i*tiles[0].shape[0]:(i+1)*tiles[0].shape[0], j*tiles[0].shape[1]:(j+1)*tiles[0].shape[1]] = tiles[i*y_tiles+j]
    return big_image

#find connected components with specified size with opencv using connectedComponentsWithStats
#does not seem to work very good, maybe we do it by hand...
#could also try with length
def get_connected_components_with_minimum_and_max_size(img: np.ndarray(dtype= np.uint8), min_size: int,max_size: int):
    """
    :param img: image where we want to find connected components. black and white immage with
                int8 rappr.
    :param min_size: minimum size of the cc
    :param max_size: maximum size of the cc
    :return:
    """
    #connecctivity shoud tell us about how connected to they have to be to be considered as one, seems to be working poorly
    connectivity = 8
    #find connected components
    num_labels, labels_im, stats, centroids = cv.connectedComponentsWithStats(img, connectivity)
    #get coordinates of connected components
    coordinates =np.array([[0,0]],dtype=np.uint64)
    print(centroids.shape)
    for i in range(1, num_labels):
        if stats[i,4] > min_size and stats[i,4]< max_size:
            coordinates=np.append(coordinates,[centroids[i,:]], axis=0)
    coordinates=coordinates[1:,:]
    return coordinates

#if a black pixel is surrounded by white pixels left right up and down we set it to white, for every pixel in the image
#to help the connected component function, maybe we can improve this
def remove_black_pixels_in_white(img: np.ndarray):
    """
    :param img: image to be better
    :return:    better immage with less black holes in white parts
    """
    #what is white? 255 or what?
    max=img.max()
    #init return image
    ret_img=np.zeros((img.shape[0],img.shape[1]))

    for i in range(1,img.shape[0]-1):
        for j in range(1,img.shape[1]-1):
            if img[i,j] == 0:
                if img[i-1,j] > 0 or img[i+1,j] > 0 or img[i,j-1] > 0 or img[i,j+1] > 0:
                    ret_img[i,j] = max
                else:
                    ret_img[i,j]=img[i,j]
    return ret_img


In [ ]:
def hard_thresholding_shit_boxes(img: np.ndarray):
    """
    Implement tresholding with a hard treshold (10000).
    Find components and box coordinates, draw the boxes with
    first coordinate of the cc.

    :param img: image to be tresholded
    """
    #sharpen image
    sharpened_img = sharpen(img)
    #hard threshold the sharpened image 10000
    threshold, hard_threshold = hard_thresholding(sharpened_img, 10000)
    #convert to uint8 s. t. coonected  component function can accept it
    hard_threshold=to_uint8(hard_threshold)
    #find coordinates of connected components, first coordinate, shit because not centered
    coordinates= find_coordinates(hard_threshold)
    #add bounding boxes to the image
    add_bounding_boxes(hard_threshold,coordinates)
    #visualize
    visualize_all_list_napari([hard_threshold,sharpened_img,img])



In [ ]:
def otsu_split_thresholding(img: np.ndarray):
    """
    Perform Otsu tresholding on sub images of 32 x 32,
    then reconstruct

    :param img:  image to be tresholded
    """
    #sharpen image
    sharpened_img=sharpen(img)
    #split
    tiles=split_into_tiles(sharpened_img,32)
    #otsu on big image
    r_big,th_big=otsu_thresholding(sharpened_img)
    #otsu on sub-images
    thresholded_tiles=[]
    for t in tiles:
        r,th=otsu_thresholding(t)
        thresholded_tiles.append(th)
    #reconstruct
    reconstructed_image=reconstruct_image(thresholded_tiles,64,47)
    #visualize
    visualize_all_list_napari([reconstructed_image,sharpened_img,img,th_big])


In [ ]:
#check if image is to be set to 0, if we have more than 400 pixels with value 0 or less then 600 pixels with value 0 we set the image to 0
def check_image(img: np.ndarray):
    """
    For every sub-image we check if its worth keeping or not

    :param img: image to be checked
    :return: bool
    """
    if np.sum(img == 0) > 200 and np.sum(img == 0) < 800: #maybe check for >0
        return True
    else:
        return False

#set pixels that are 0 to 255(white)
def set_one(img):
    h=img
    h[h ==0 ] = 255
    return h

#set pixels that are 255 to zero (black)
def set_zero(img):
    h=img
    h[h>0]=0
    return h


In [ ]:
def otsu_cleaned_white_split_thresholding(img):
    """
    Performed otsu on an immage that was cleaned in white

    :param img: image to be tresholded
    :return:    tresholded clean image
    """
    #sharpen image
    sharpened_img= sharpen(img)
    #split
    tiles=split_into_tiles(sharpened_img,32)
    #treshold
    thresholded_tiles=[]
    for t in tiles:
        r,th_image=otsu_thresholding(t)
        thresholded_tiles.append(th_image)
    #reconstruct
    reconstructed_image=reconstruct_image(thresholded_tiles,64,47)
    #clean
    cleaned_tiles=[]
    for tl in thresholded_tiles:
           if check_image(tl):
                m=set_one(tl)
                cleaned_tiles.append(m)
           else:
               cleaned_tiles.append(tl)
    #reconstruct
    reconstructed_clean_image=reconstruct_image(cleaned_tiles,64,47)
    #visualize
    visualize_all_list_napari([reconstructed_image,reconstructed_clean_image,sharpened_img,img])
    return reconstructed_clean_image

cleaned_im_white=otsu_cleaned_white_split_thresholding(img)


In [ ]:
def otsu_cleaned_black_split_thresholding(img):
    """
    Performed otsu on an immage that was cleaned in white

    :param img: image to be tresholded
    :return:    tresholded clean image
    """

    sharpened_img= sharpen(img)

    tiles=split_into_tiles(sharpened_img,32)

    thresholded_tiles=[]
    for t in tiles:
        r,th_image=otsu_thresholding(t)
        thresholded_tiles.append(th_image)
    reconstructed_image=reconstruct_image(thresholded_tiles,64,47)

    cleaned_tiles=[]
    for tl in thresholded_tiles:
           if check_image(tl):
                m=set_zero(tl)                      #clean in black only thing that changes
                cleaned_tiles.append(m)
           else:
               cleaned_tiles.append(tl)

    reconstructed_clean_image=reconstruct_image(cleaned_tiles,64,47)

    visualize_all_list_napari([reconstructed_image,reconstructed_clean_image,sharpened_img,img])

    return reconstructed_clean_image

cleaned_im_black=otsu_cleaned_black_split_thresholding(img)

In [ ]:

cleaned_im_black=remove_black_pixels_in_white(cleaned_im_black)
cleaned_im_black_int=to_uint8(cleaned_im_black)

new_coordinates=get_connected_components_with_minimum_and_max_size(cleaned_im_black_int, 0,100000)


new_coordinates=new_coordinates.astype(int)

add_bounding_boxes(cleaned_im_black,new_coordinates)




images = [img, cleaned_im_black]

visualize_all_list_napari(images)

